In [1]:
import requests, time, os
from datetime import datetime
import xml.etree.ElementTree as ET
import xml_to_df
import pandas as pd

In [2]:
ids_pvp = pd.read_csv("data/ListaProposicoesVotadasPlenario_2013_2024.csv") 

In [3]:
# example

url_vpi = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicaoPorID'
id = 1672576
params = {
    'idProposicao': id
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [4]:
response_test = requests.get(url_vpi, params=params, headers=headers)

In [5]:
root = ET.fromstring(response_test.text)
xml_text = ET.tostring(root, encoding='unicode')
print(xml_text)

<proposicao>
  <Votacoes>
    <Votacao Resumo="Rejeitado o Requerimento. Sim: 27; não: 394; total: 421.&#13;&#10;" Data="28/8/2024" Hora="18:41" ObjVotacao="REQUERIMENTO DE RETIRADA DE PAUTA" codSessao="17500" nomeProposicao="PL 1829/2019 (Nº Anterior: PL 2724/2015)">
      <orientacaoBancada>
        <bancada Sigla="Bl UniPpFdrPsdbCid..." orientacao="Não            " />
        <bancada Sigla="Bl MdbPsdRepPode" orientacao="Não            " />
        <bancada Sigla="PL" orientacao="Não            " />
        <bancada Sigla="Fdr PT-PCdoB-PV" orientacao="Não            " />
        <bancada Sigla="PSB" orientacao="Não            " />
        <bancada Sigla="Fdr PSOL-REDE" orientacao="Sim            " />
        <bancada Sigla="Repr.Novo" orientacao="Sim            " />
        <bancada Sigla="Maioria" orientacao="Não            " />
        <bancada Sigla="Minoria" orientacao="Liberado       " />
        <bancada Sigla="Oposição" orientacao="Liberado       " />
        <bancada Sigla="

In [17]:
responses_vpi = {}
dfs_vpi = {}

lista_dados = []

for id in ids_pvp.codProposicao.unique():
    
    params = {
        'idProposicao': id
    }

    try:
        response = xml_to_df.get_data(url_vpi, params, headers)
        root = ET.fromstring(response.text)

        for votacao in root.findall(".//Votacao"):
            dados = {}
            for attr in votacao.attrib:
                dados[f'{attr.lower()}'] = votacao.get(attr)
            
            # Iterar sobre orientações de bancadas
            for bancada in votacao.findall(".//bancada"):
                for attr in bancada.attrib:
                    dados[f'bancada_{attr.lower()}'] = bancada.get(attr)

            # Iterar sobre os votos dos deputados
            for deputado in votacao.findall(".//Deputado"):
                for attr in deputado.attrib:
                    dados[f'deputado_{attr.lower()}'] = deputado.get(attr)

            lista_dados.append(dados)

        if response.status_code != 200:
            print(f'Erro ao extrair dados de votação do PL de id = {id}')
            continue
        
        df = pd.DataFrame(lista_dados)
        dfs_vpi[id] = df
        responses_vpi[id] = response
    
    except Exception as e:
        print(f'Erro ao processar dados de votação do PL de id = {id}: {e}')

merged_df = pd.concat(dfs_vpi.values(), ignore_index=True)
xml_to_df.save_csv(merged_df, 'votacoes_proposicoes_2013_2024.csv')

In [10]:
response = requests.get(url_vpi, params=params, headers=headers)
root = ET.fromstring(response.text)

In [12]:
for votacao in root.findall(".//Deputado"):
    for attr in votacao.attrib:
        print(attr, votacao.get(attr))

Nome Defensor Stélio Dener
ideCadastro 220539
Partido Republican
UF RR
Voto Não            
Nome Duda Ramos
ideCadastro 220540
Partido MDB       
UF RR
Voto Não            
Nome Augusto Coutinho
ideCadastro 160665
Partido Republican
UF PE
Voto Não            
Nome Carlos Veras
ideCadastro 204426
Partido PT        
UF PE
Voto Não            
Nome Clodoaldo Magalhães
ideCadastro 88256
Partido PV        
UF PE
Voto Não            
Nome Coronel Meira
ideCadastro 220666
Partido PL        
UF PE
Voto Não            
Nome Fernando Monteiro
ideCadastro 92699
Partido PP        
UF PE
Voto Não            
Nome Fernando Rodolfo
ideCadastro 204427
Partido PL        
UF PE
Voto Não            
Nome Eriberto Medeiros
ideCadastro 220663
Partido PSB       
UF PE
Voto Não            
Nome Guilherme Uchoa
ideCadastro 220664
Partido PSB       
UF PE
Voto Não            
Nome Iza Arruda
ideCadastro 220670
Partido MDB       
UF PE
Voto Não            
Nome Lucas Ramos
ideCadastro 220662
Partido PSB       
